In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [17]:
start_date = '20221226'
end_date = '20230219' 

'20220801' to '20220828' 
'20220829' to '20221030'
'20221031' to '20221225'
'20221226' to '20230219'

In [18]:
query = f"""

with base as (
    select 
        date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') order_date, 
        date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_week, 
        city_name, order_type, order_id, customer_id,captain_id,
        distance_final_distance,
        service_obj_service_name service,
        ride_distance*0.8 ride_distance_threshold,
        spd_fraud_flag,ride_time,ride_distance,
        amount,amount_breakup_base_fare_total,actual_price
        
    from orders.order_logs_snapshot
    where order_status = 'dropped' --and event_type = 'dropped'
        and yyyymmdd BETWEEN '{start_date}' and '{end_date}'
        and service_obj_service_name = 'Auto'
),

segment as (
select * 
from 
(
select 
    date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') order_date, 
    date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_wk, 
    captain_id, 
    case 
    when auto_pc_segment like 'HP_D%' then '3. HP Daily'
    when auto_pc_segment like 'HP_i%' then '4. HP Intra/Inter'

    when auto_pc_segment like 'MP_D%' then '5. MP Daily'
    when auto_pc_segment like 'MP_i%' then '6. MP Intra/Inter'

    when auto_pc_segment like 'LP_D%' then '7. LP Daily'
    when auto_pc_segment like 'LP_i%' then '8. LP Intra/Inter'
    
    when auto_pc_segment like 'UHP_D%' then '1. UHP Daily'
    when auto_pc_segment like 'UHP_i%' then '2. UHP Intra/Inter'
    
    end segment,
    row_number() over(partition by date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d'), captain_id order by yyyymmdd desc) seq_no
from datasets.captain_metrics_segments
where
    yyyymmdd BETWEEN '{start_date}' and '{end_date}'
    and auto_pc_segment is not null 
    -- and day_of_week(cast (date_parse(yyyymmdd, '%Y%m%d') as date)) = 7
    and captain_id in (select distinct captain_id from base)
 ) where seq_no = 1    
),

total_partial_offline_ride as (

select 
    order_week,
    service,
    count(distinct order_id) total_orders,
    count(distinct case when distance_final_distance <= ride_distance_threshold then order_id end) as total_partial_orders,
    count(distinct case when ride_distance <= 5.0 and distance_final_distance <= ride_distance_threshold then order_id end) as short_partial_orders,
    count(distinct case when ride_distance > 5.0 and distance_final_distance <= ride_distance_threshold then order_id end) as long_partial_orders
from base
group by 1,2
),


partial_offline_ride as (
select 
    order_week,
    service,
    coalesce(segment,'0. New') segment,
    count(distinct base.customer_id) uniq_customer,
    count(distinct base.captain_id) uniq_captain,
    count(distinct order_id) as total_orders,
    count(distinct case when distance_final_distance <= ride_distance_threshold then order_id end) as total_partial_orders,
    count(distinct case when distance_final_distance <= ride_distance_threshold then base.customer_id end) partial_uniq_customer,
    count(distinct case when distance_final_distance <= ride_distance_threshold then base.captain_id end) partial_uniq_captain,
    count(distinct case when ride_distance <= 5.0 and distance_final_distance <= ride_distance_threshold then order_id end) as short_partial_orders,
    count(distinct case when ride_distance > 5.0 and distance_final_distance <= ride_distance_threshold then order_id end) as long_partial_orders,
    sum( case when distance_final_distance <= ride_distance_threshold then amount_breakup_base_fare_total end) as actual_fare,
    sum( case when distance_final_distance <= ride_distance_threshold then actual_price end) as estimated_fare,
    sum( case when distance_final_distance <= ride_distance_threshold then 0.2*actual_price end) as expected_commission
from base
left join segment s on order_week = order_wk and s.captain_id = base.captain_id
group by 1,2,3
)

select 
    a.order_week,
    -- a.service,
    a.segment,
    a.total_orders,
    a.total_partial_orders,
    try(a.total_partial_orders*100.00/b.total_partial_orders) "partial_orders_%",
    a.expected_commission,
    a.short_partial_orders,
    a.long_partial_orders,
    -- try(a.short_partial_orders*100.00/b.total_partial_orders) short_partial_distr,
    -- try(a.long_partial_orders*100.00/b.total_partial_orders) long_partial_distr,
    try(a.short_partial_orders*100.00/b.short_partial_orders) "short_partial_%",
    try(a.long_partial_orders*100.00/b.long_partial_orders) "long_partial_%" ,
    a.actual_fare,
    a.estimated_fare,
    a.uniq_captain,
    a.partial_uniq_captain,
    a.uniq_customer,
    a.partial_uniq_customer
    
from partial_offline_ride a
left join total_partial_offline_ride b on a.order_week = b.order_week and a.service = b.service

order by 1 desc,2

"""

In [19]:
df_code = pd.read_sql(query, conn)
df_code 

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2023-02-13,0. New,5074,490,0.55,6153.8,288,202,0.50,0.62,11925.0,30769.0,1861,386,4968,460
1,2023-02-13,1. UHP Daily,150060,8675,9.70,87283.2,7755,920,13.58,2.85,235050.0,436416.0,1492,1343,109567,8024
2,2023-02-13,2. UHP Intra/Inter,44162,2429,2.72,24784.4,2175,254,3.81,0.79,65645.0,123922.0,713,539,37336,2280
3,2023-02-13,3. HP Daily,338529,17070,19.09,235844.0,12174,4896,21.32,15.14,416916.0,1179220.0,7533,5830,278747,16307
4,2023-02-13,4. HP Intra/Inter,94738,4921,5.50,67747.2,3641,1280,6.38,3.96,118319.0,338736.0,3391,2033,88378,4839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-12-26,4. HP Intra/Inter,69963,1206,3.23,17553.8,647,559,4.20,2.55,30292.0,87769.0,2615,849,65447,1204
68,2022-12-26,5. MP Daily,566879,11414,30.57,181619.4,4904,6510,31.82,29.69,285640.0,908097.0,32937,8739,451890,11220
69,2022-12-26,6. MP Intra/Inter,223432,4543,12.17,72314.6,2038,2505,13.22,11.43,111076.0,361573.0,21962,3661,201373,4506
70,2022-12-26,7. LP Daily,170999,3961,10.61,65629.4,1070,2891,6.94,13.19,98168.0,328147.0,21118,3157,154772,3939


In [6]:
df1 = df_code
df1.head()

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2022-08-22,0. New,76457,2153,5.78,29299.0,1165,988,6.55,5.08,65172.0,146495.0,11681,1778,52027,2046
1,2022-08-22,1. UHP Daily,164649,2608,7.00,38423.6,1941,667,10.91,3.43,95375.0,192118.0,1962,1224,120087,2447
2,2022-08-22,2. UHP Intra/Inter,28637,490,1.32,7037.6,376,114,2.11,0.59,14496.5,35188.0,552,256,23413,475
3,2022-08-22,3. HP Daily,356227,6396,17.17,113192.2,3536,2860,19.88,14.69,235161.0,565961.0,8021,3879,273284,6234
4,2022-08-22,4. HP Intra/Inter,75854,1446,3.88,24410.6,878,568,4.94,2.92,48818.5,122053.0,2911,974,69254,1421


In [12]:
df2 = df_code
df2.head()

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2022-10-24,0. New,1239,195,0.68,2436.6,89,106,0.74,0.63,4767.0,12183.0,214,129,1233,194
1,2022-10-24,1. UHP Daily,75313,1035,3.59,12845.8,703,332,5.87,1.97,31158.0,64229.0,1163,595,65339,1019
2,2022-10-24,2. UHP Intra/Inter,21384,284,0.99,3411.6,198,86,1.65,0.51,8918.0,17058.0,458,178,19744,281
3,2022-10-24,3. HP Daily,265256,4216,14.62,60006.8,2256,1960,18.83,11.63,115188.0,300034.0,6725,2662,210709,4145
4,2022-10-24,4. HP Intra/Inter,65422,1042,3.61,14750.4,592,450,4.94,2.67,28879.0,73752.0,2604,705,60632,1028


In [16]:
df3 = df_code
df3.head()

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2022-12-19,0. New,1645,267,0.68,2867.0,93,174,0.58,0.74,6230.0,14335.0,260,174,1620,260
1,2022-12-19,1. UHP Daily,153023,1908,4.84,21506.2,1408,500,8.80,2.14,51859.0,107531.0,1625,975,109883,1855
2,2022-12-19,2. UHP Intra/Inter,31453,344,0.87,3775.8,270,74,1.69,0.32,9473.0,18879.0,571,218,26134,334
3,2022-12-19,3. HP Daily,317378,5236,13.28,72443.0,2798,2438,17.49,10.41,132418.0,362215.0,6634,3172,254122,5114
4,2022-12-19,4. HP Intra/Inter,68939,1146,2.91,16217.0,608,538,3.80,2.30,28401.0,81085.0,2432,770,64772,1139


In [20]:
df4 = df_code
df4.head()

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2023-02-13,0. New,5074,490,0.55,6153.8,288,202,0.50,0.62,11925.0,30769.0,1861,386,4968,460
1,2023-02-13,1. UHP Daily,150060,8675,9.70,87283.2,7755,920,13.58,2.85,235050.0,436416.0,1492,1343,109567,8024
2,2023-02-13,2. UHP Intra/Inter,44162,2429,2.72,24784.4,2175,254,3.81,0.79,65645.0,123922.0,713,539,37336,2280
3,2023-02-13,3. HP Daily,338529,17070,19.09,235844.0,12174,4896,21.32,15.14,416916.0,1179220.0,7533,5830,278747,16307
4,2023-02-13,4. HP Intra/Inter,94738,4921,5.50,67747.2,3641,1280,6.38,3.96,118319.0,338736.0,3391,2033,88378,4839


In [21]:
frames = [df1,df2,df3,df4]
results = pd.concat(frames)

In [22]:
results

,order_week,segment,total_orders,total_partial_orders,partial_orders_%,expected_commission,short_partial_orders,long_partial_orders,short_partial_%,long_partial_%,actual_fare,estimated_fare,uniq_captain,partial_uniq_captain,uniq_customer,partial_uniq_customer
0,2022-08-22,0. New,76457,2153,5.78,29299.0,1165,988,6.55,5.08,65172.0,146495.0,11681,1778,52027,2046
1,2022-08-22,1. UHP Daily,164649,2608,7.00,38423.6,1941,667,10.91,3.43,95375.0,192118.0,1962,1224,120087,2447
2,2022-08-22,2. UHP Intra/Inter,28637,490,1.32,7037.6,376,114,2.11,0.59,14496.5,35188.0,552,256,23413,475
3,2022-08-22,3. HP Daily,356227,6396,17.17,113192.2,3536,2860,19.88,14.69,235161.0,565961.0,8021,3879,273284,6234
4,2022-08-22,4. HP Intra/Inter,75854,1446,3.88,24410.6,878,568,4.94,2.92,48818.5,122053.0,2911,974,69254,1421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-12-26,4. HP Intra/Inter,69963,1206,3.23,17553.8,647,559,4.20,2.55,30292.0,87769.0,2615,849,65447,1204
68,2022-12-26,5. MP Daily,566879,11414,30.57,181619.4,4904,6510,31.82,29.69,285640.0,908097.0,32937,8739,451890,11220
69,2022-12-26,6. MP Intra/Inter,223432,4543,12.17,72314.6,2038,2505,13.22,11.43,111076.0,361573.0,21962,3661,201373,4506
70,2022-12-26,7. LP Daily,170999,3961,10.61,65629.4,1070,2891,6.94,13.19,98168.0,328147.0,21118,3157,154772,3939


In [24]:
results.to_csv("partial_auto.csv", index = False)